# Load data from Zendesk Support API to Weaviate
Here is a guide on how to use the [Zendesk dlt source](https://dlthub.com/docs/dlt-ecosystem/verified-sources/zendesk) to load support data into Weaviate with just a few lines of code.

**Note:** to run this notebook, you need to set up [credentials](https://dlthub.com/docs/general-usage/credentials) for both Zendesk and Weaviate. Learn how to configure the source and destination in the dlt documentation:
- [Setting up credentials for the Zendesk source](https://dlthub.com/docs/dlt-ecosystem/verified-sources/zendesk#get-api-credentials)
- [Configuring the Weaviate destination](https://dlthub.com/docs/dlt-ecosystem/destinations/weaviate)

## Install and init dlt

In [ ]:
%%capture
!pip install dlt[weaviate]

## Install Zendesk dlt source

In [ ]:
!dlt init zendesk weaviate # or salesforce, or jira, or notion, and more

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
No files to update, exiting


## Load Data

In [ ]:
import dlt
from dlt.destinations.weaviate import weaviate_adapter

from zendesk import zendesk_support

pipeline = dlt.pipeline(
    pipeline_name="zendesk_pipeline",
    destination="weaviate",
)

zendesk_source = zendesk_support(load_all=False)
tickets = zendesk_source.tickets

info = pipeline.run(
    weaviate_adapter(
        tickets,
        vectorize=["subject", "description"],
    ),
)

In [ ]:
print(info)

Pipeline zendesk_pipeline completed in 10.23 seconds
1 load package(s) were loaded to destination weaviate and into dataset None
The weaviate destination used https://demo-colab-ng73n7k8.weaviate.network location to store data
Load package 1693572743.108808 is LOADED and contains no failed jobs


And we're done. Let's check out the data.

## Query Data

You need to replace the `url` and `api_key` below with your Weaviate instance credentials.

In [ ]:
import os
import pandas as pd
import weaviate

def response_to_table(response, class_name, columns):
    df = pd.DataFrame(response['data']['Get'][class_name], columns=columns)
    display(df)


client = weaviate.Client(
    url='https://YOUR_WEAVIATE_URL',
    auth_client_secret=weaviate.AuthApiKey(
        api_key='YOUR_WEAVIATE_API_KEY'
    ),
)

response = (
    client.query
    .get("Tickets", ["subject", "description", "created_at"])
    .do()
)

response_to_table(response, 'Tickets', ["subject", "description", "created_at"])

,subject,description,created_at
0,Another ticket,There is a problem that needs fixing,2023-07-18T17:24:53Z
1,Issues with API,"We are having trouble with the API, it is retu...",2023-09-01T11:18:25Z
2,Billing Inquiry,I noticed a charge on my credit card that I do...,2023-09-01T11:16:52Z
3,Unable to update profile,I am trying to update my profile details but k...,2023-09-01T11:17:45Z
4,Unable to Cancel Subscription,I have been trying to cancel my subscription b...,2023-09-01T11:19:25Z
5,What is this ticket,ABCDEF,2023-07-18T17:23:42Z
6,Difficulty in accessing reports,I have been trying to access the reports secti...,2023-09-01T11:18:03Z
7,Trouble with Two-Factor Authentication,I recently enabled two-factor authentication o...,2023-09-01T11:17:20Z
8,Mobile App Crash,The mobile app keeps crashing every time I try...,2023-09-01T11:18:53Z
9,Login Issues,▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇,2023-03-14T10:52:28Z
